# Домашнее задание - линейная регрессия

### Работа с признаками (8 баллов)

Скачайте датасет из материалов к уроку или по ссылке https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv 


Описание признаков:

* Agency — название страхового агентства
* Agency Type — тип страхового агентства
* Distribution Channel — канал продвижения страхового агентства
* Product Name — название страхового продукта
* Duration — длительность поездки (количество дней)
* Destination — направление поездки
* Net Sales — сумма продаж 
* Commission (in value) — комиссия страхового агентства
* Gender — пол застрахованного
* Age — возраст застрахованного

Ответ:
* Claim — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

Обработайте пропущенные значения и примените написанные функции onehot_encode() и minmax_scale().

**Подсказка**: маску для категориальных признаков можно сделать фильтром cat_features_mask = (df.dtypes == "object").values

In [ ]:
# < напишите код здесь > 
import pandas as pd
import numpy as np
from sklearn import impute

def minmax_scale(X: np.array):
    if np.all(X.max(axis=0) == X.min(axis=0)):
        return (X - X.min(axis=0)).astype(float)
    X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
    return X_std


def onehot_encoding(x: np.array):
    n = np.size(x)
    x_uniq = np.sort(np.unique(x))
    m = np.size(x_uniq)
    new_array = np.zeros((n, m), int)
    for i in range(n):
        new_array[i, np.where(x_uniq == x[i])[0][0]] = 1

    return new_array


df = pd.read_csv('travel insurance.csv')
cat_features_mask = ((df.dtypes == "object")).values # Категориальные признаки
num_columns = df.columns[~cat_features_mask] # Числовые признаки
cat_columns = df.columns[cat_features_mask] #Категориальные признаки
cat_columns = np.delete(cat_columns, np.where(cat_columns == 'Claim')) # оставили только признаки
# print(cat_columns)
# print(num_columns)

# получаем столбцы разных признаков
df_num = df[num_columns]
df_cat = df[cat_columns]

# print(df_cat)
# print(df_num)

# заполнение средним в числовых
replacer = impute.SimpleImputer(strategy="mean")
df_num_no_ = pd.DataFrame(data = replacer.fit_transform(df_num), columns = df_num.columns)

# заполнение "" в категориальных признаках и удаление столбца Claim, который не является признаком в данной задаче
df_cat_no_ = df_cat.fillna("")

# df_num_no_.head()
# df_cat_no_.head()

# применяем onehot_encode()
encoded_cats = np.hstack([onehot_encoding(df_cat_no_[col]) for col in cat_columns])
# print(encoded_cats)

# применяем minmax_scale()
scaled_nums = minmax_scale(df_num_no_)
# print(df_num_no_)
# print(scaled_nums)


Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Destination', 'Gender'],
      dtype='object')
Index(['Duration', 'Net Sales', 'Commision (in value)', 'Age'], dtype='object')


Подробный анализ и подготовка датасета часто помогают улучшить качество модели. Ниже представлено несколько идей преобразований. Вы можете применить одно или несколько из этих преобразований (а можете не применять), чтобы помочь будущей модели. 

1. Посмотрите на количественные признаки. Возможно, в некоторых признаках есть выбросы - значения, которые сильно выбиваются. Такие значения полезно удалять. Советуем присмотреться к колонке Duration)

2. Можно заметить, что one hot encoding сильно раздувает количество столбцов. Радикальное решение - можно попробовать выбросить все категориальные признаки из датасета.

3. Если все-таки оставляете категориальные признаки, то подумайте, как уменьшить количество столбцов после one hot encoding. Признаки с большим количеством значений (Duration - 149! разных стран) можно удалить или попробовать сгруппировать некоторые значения.

4. Downsampling. Датасет достаточно большой, разница в классах огромная. Можно уменьшить число наблюдений с частым ответом.

In [63]:
# совместим категориальные и численные признаки
final_data = np.hstack([encoded_cats, scaled_nums])

# Извлечение Ответов
y = df["Claim"].apply(lambda x: 1 if x == "Yes" else 0).values
X = final_data


### Применение линейной регрессии (10 баллов)

Это задача классификации, но её можно решить с помощью линейной регрессии, если округлять предсказанный ответ до целого и выбирать ближайший по значению ответ из множества {0, 1}.

Вынесите признак 'Claim' в вектор ответов и разделите датасет на обучающую и тестовую выборку в соотношении 80 к 20. Зафиксируйте random_state.

**Подсказка:** быстро перевести Yes/No в 1/0 можно так - np.where(df['Claim'] == 'Yes', 1,0)

In [65]:
# разделение на test/train
from sklearn.model_selection import train_test_split


rand_stat = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = rand_stat)

Найдите аналитическое решение для обучающей выборки: обычное и регуляризацией l2. 

In [74]:
# посчитайте аналитическое решение
def regress(X, y):
    return np.linalg.pinv(X.T @ X) @ X.T @ y


X_train_1 = np.hstack([np.ones((X_train.shape[0], 1)), X_train])  # добавляем столбец единичных значений
W_normal = regress(X_train_1, y_train)
# print(W_normal)

In [75]:
# посчитать аналитическое решение с регуляризацией
def L2_regression(X, y, lambda_reg):
    n_features = X.shape[1]
    I = np.eye(n_features) 
    return np.linalg.inv(X.T @ X + lambda_reg * I) @ X.T @ y


lambda_reg = 1  # Пример значения коэффициента регуляризации
W_l2 = L2_regression(X_train_1, y_train, lambda_reg)
# print(W_l2)

Постройте модель LinearRegression, примените к тестовой выборке и посчитайте MSE (можно использовать библиотеку sklearn)


In [76]:
# обучите модель линейной регрессии LinearRegression на обучающей выборке, примените к тестовойfrom sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [80]:
# посчитайте MSE, предварительно округлив предсказанные ответы до целого
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {round(mse)}')

Mean Squared Error (MSE): 0


### Вывод (1 балла)

Напишите краткий вывод по заданию (достаточно пары предложений). Расскажите, какие способы предобработки данных вы выбрали и почему. Насколько хороша ваша модель?

Мы использовали преобразования onehot_encode для пролучения матриц, обычные числовые значения мы сделали поближе к друг другу с помощью minmax_scale. Преварительно убрали Claim из списка столбцов. Затем разделили с sklearn split на выборки и посчитали аналитическое решение и решение l2 с лямбой = 1, ну как в на семинаре, далее посчитали метрику MSE и получили наше отклонение. 